In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
import statsmodels.formula.api as sm
import patsy
import itertools
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, auc
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold, GridSearchCV,learning_curve
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer 
from sklearn.pipeline import Pipeline

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yesplum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [213]:
import glob, os     
extension = 'csv' 
all_filenames = [i for i in glob.glob('*.{}'.format(extension))] 
print(all_filenames)

#combine all files in the list 

df_raw = pd.concat([pd.read_csv(f, encoding='unicode escape',skiprows=0) for f in all_filenames ]) 
df_raw.reset_index(inplace=True) 
df_raw = df_raw.drop(columns=['index','Unnamed: 0']) 

['DS_27Mar2020a.csv', 'DS_27Mar2020b.csv', 'ML_27Mar2020b.csv', 'ML_27Mar2020a.csv', 'AI_26Mar2020.csv']


In [214]:
df_raw.shape

(1091, 11)

In [239]:
def data_clean(df):
    
    #remove duplicate based on Job ID
    job_clean = df_raw.drop_duplicates(subset='Job_Id', keep='first')
    #drop Salary_Type due to only one unique value 'Monthly'
    job_clean = job_clean.drop(columns='Salary_Type')
    #remove job without title
    job_no_title = job_clean['Job_Title'] == ''
    job_clean = job_clean[~job_no_title]
    #remove row with all NaN value
    job_clean[job_clean.isnull().any(axis=1)]
    job_clean = job_clean.dropna()
    
    #perform data cleaning on every row and columms
    clean_list = "(\[|\]|b'|Requirements|'|amp;|xa0|\\\|xe2x80x93|\\n|div class=|div class=|span class=|dib|lh-solid|/span|f5-5 i fw4 gray|f5 fw4 ph1|<|>|/div|\")"
    for col in job_clean.columns.difference(['Requirements']):
        job_clean[col]=job_clean[col].str.replace(clean_list, "")

    #space remain for Requirements column    
    job_clean['Requirements']=job_clean['Requirements'].str.replace(clean_list, " ")

    clean_list2 = "(Ã¢Â€Â|Ã¢Â€Âœ|Ã¢Â€Â™|ïƒ¼ Â|ïƒ¨|ïƒ¼|Â Â Â Â Â|Â|â€“|â€™|â€œâ€|Â|Ã¢Â€Â™️)"
    job_clean['Requirements']=job_clean['Requirements'].str.replace(clean_list2, "")
    
    #further remove job with same data from all columns
    job_clean = job_clean.drop_duplicates(subset=job_clean.columns, keep='first') 
    
    
    #further filter on job title with specific keywords
    title_key = ['DATA', 'MACHINE','ANALYST','MACHINE LEARNING','ANALYTICS', "SCIENCE", '4.0','APPLICATION'
             'DEEP LEARNING','RESEARCH','NLP', 'ARTIFICIAL', "INTELLIGENT", 'AI', 'SCIENTIST','SYSTEM'
             'Industry', 'IOT', 'FINANCE', 'FINTECH', 'SOFTWARE', 'ENGINEER', 'ENGINEERING','PROFESSOR'
             'BUSINESS', 'DEVELOPER', 'INDUSTRIAL','AUTOMATION', 'CLOUD','SOLUTION','ARCHITECT',
             'MANAGER','VP','PRESIDENT', 'TECHNOLOGY', 'SPECIALIST', 'TECHNICAL','LEAD','TECHNOLOGIST']
    key = '|'.join(title_key)
    data_job = job_clean['Job_Title'].str.upper().str.contains(key)
    job_clean = job_clean[data_job]

    #remove job title with unwanted keywords
    title_key = ['PHYSIOTHERAPIST','ACCOUNT','AUDIT','COUNSEL','EXECUTIVE','SALES','GENERAL','MARKET',
                 'ELECTRICAL','BUSINESS','ADMIN','CUSTOMER','OFFICER','OPERATION', 'MECHANICAL','CHEMICAL',
                 'COORDINATOR','LECTURER','TECHNICIAN']
    key = '|'.join(title_key)
    non_data_job = job_clean['Job_Title'].str.upper().str.contains(key)
    job_clean = job_clean[~non_data_job]
    
    #remove job with multiple category
    cat_list = "(/|and)"
    job_clean['Category']=job_clean['Category'].str.replace(cat_list, ",")
    job_clean['Cat_num'] = job_clean['Category'].str.count(',')
    
    multiple_cat = job_clean['Cat_num']>5
    job_clean = job_clean[~multiple_cat]
    job_clean = job_clean.drop(columns='Cat_num')
    
    #remove job with no or multiple seniority
    senior_rule = (job_clean['Seniority'].str.count(',')>=1) | (job_clean['Seniority']=='')
    job_clean = job_clean[~senior_rule]

    
    #remove job cat with specific keywords
    rare_cat_key = ['HUMAN','SOCIAL','THERAPY','TAXATION','CUSTOMER','INTERIOR', 'ADMIN','BUILDING',
                    'SECRETARIAL','INVESTIGATION', 'AUDITING', 'ENVIRONMENT','SALES', 'MARKETING',
                    'ADVERTISING','CONSTRUCTION', 'DESIGN','LEGAL','HOSPITALITY','PROFESSIONAL']
    key = '|'.join(rare_cat_key)
    rare_cat = job_clean['Category'].str.upper().str.contains(key)
    job_clean = job_clean[~rare_cat]
    
    #remove row without salary
    no_salary = job_clean['Salary_Range'].str.contains('Salary undisclosed')
    df_salary = job_clean[~no_salary]
    df_no_salary = job_clean[no_salary]
    df_salary = df_salary.reset_index(drop=True)
    
    req_empty = []

    for i in range (len(df_salary)):
    
        if((len(df_salary['Requirements'][i]))<5):
            req_empty.append(i)
           
    #clean & remove row without requirements
    df_salary['Requirements']=df_salary['Requirements'].str.replace('(\n)', "")
    df_salary = df_salary.drop(req_empty)
    df_salary = df_salary.reset_index(drop=True)

    return df_salary

In [240]:
df = data_clean(df_raw)
df.shape

(458, 10)

In [241]:
def salary_feature(df):
    
    #extract salary columns due to contain multiple information
    salary_range = df["Salary_Range"].str.split("to", n = 2, expand = True) 

    #Give columns name to the dataframe
    salary_range = salary_range.rename({0:'Min_Salary',1:'Max_Salary'}, axis='columns')

    #removed $ and , from salary 
    for col in salary_range.columns:
        salary_range[col]=salary_range[col].str.replace('(\$|,)', '')

    #convert from ojbect to float for statistical infomation
    salary_range['Min_Salary'] = salary_range['Min_Salary'].astype('float64')
    salary_range['Max_Salary'] = salary_range['Max_Salary'].astype('float64')
    
    #concat min_max salary dataframe with salary range dataframe
    df_salary1 = pd.concat([df, salary_range], axis=1)
    df_salary1 = df_salary1.drop(columns='Salary_Range')  
    
    #create a condition to check for high outliers
    abovemean_min = round(10*np.mean(df_salary1['Min_Salary']),0)
    abovemean_max = round(10*np.mean(df_salary1['Max_Salary']),0)
    
    #convert yearly salary into monthly salary

    df_salary1['Min_Salary'] = np.where((df_salary1['Min_Salary'] > abovemean_min),
                                    round((df_salary1['Min_Salary']/12),0), df_salary1['Min_Salary'])

    df_salary1['Max_Salary'] = np.where((df_salary1['Max_Salary'] > abovemean_min),
                                    round((df_salary1['Max_Salary']/12),0), df_salary1['Max_Salary'])
    
    #drop unrealistic min and max monthly salary range (which is more than 10 times)
    min_max_abnormal = (df_salary1['Max_Salary']>10*df_salary1['Min_Salary'])
    df_salary1 = df_salary1[~min_max_abnormal]
    
    #drop job with max salary less than 2500, assuming data entry/admin/operator job
    low_sal = ((df_salary1['Min_Salary']<=1800) | (df_salary1['Max_Salary']<=2500))
    df_salary1 = df_salary1[~low_sal]
    
    #create new feature for average salary
    df_salary1['Avg_Salary'] = (df_salary1['Min_Salary'] + df_salary1['Max_Salary']) / 2
    
    #drop job with outlier salary
    salary_outlier = ((df_salary1['Avg_Salary']>20000) | (df_salary1['Avg_Salary']<3000))
    df_salary1 = df_salary1[~salary_outlier]
    
    #bin salary into 4 groups:
    #3000 to 5500 - Low
    #5500 to 8000 - Med
    #8000 and above - High

    bins = [3000, 5500, 8000, np.inf]
    names = ['Low', 'Med', 'High']

    df_salary1['Salary_range'] = pd.cut(df_salary1['Avg_Salary'], bins, labels=names)
    df_salary1 = df_salary1.reset_index(drop=True)
    
    return df_salary1

In [242]:
df1 = salary_feature(df)
df1.shape

(457, 13)

In [243]:
def emp_type(df):

    
    #remove others employment type
    type_key = ['PART TIME','TEMPORARY','INTERNSHIP','FLEXI','FREELANCE']
    key = '|'.join(type_key)
    non_type = df['Emp_Type'].str.upper().str.contains(key)
    df = df[~non_type]
    df = df.reset_index(drop=True)

    #consolidate employment type
    consolidate = "(Full Time|Permanent, Full Time)"
    df['Emp_Type']=df['Emp_Type'].str.replace(consolidate, "Permanent")

    consolidate = "(Contract, Full Time)"
    df['Emp_Type']=df['Emp_Type'].str.replace(consolidate, "Contract")

    consolidate = "(Contract, Permanent, Full Time)"
    df['Emp_Type']=df['Emp_Type'].str.replace(consolidate, "Cont_Perm")
    
    return df

In [244]:
df1 = emp_type(df1)
df1.shape

(453, 13)

In [245]:
def seniority(df):
    
    #consolidate seniority from 9 groups to 4 groups
    
    df['Seniority'] = np.where((df['Seniority'] == 'Fresh/entry level') | (df['Seniority'] == 'Non-executive') | (df['Seniority'] == 'Junior Executive'),
                                 'Jr Executive', df['Seniority'])

    df['Seniority'] = np.where((df['Seniority'] == 'Executive') | (df['Seniority'] == 'Senior Executive'),
                                 'Sr Executive', df['Seniority'])

    df['Seniority'] = np.where((df['Seniority'] == 'Manager') | (df['Seniority'] == 'Middle Management') | (df['Seniority'] == 'Senior Management'),
                                 'Management', df['Seniority'])
    
    return df

In [246]:
df1 = seniority(df1)
df1.shape

(453, 13)

In [247]:
def cat_name(df):
    
    stacked = pd.DataFrame(df['Category'].str.split(',').tolist()).stack()
    cat_count = pd.DataFrame(stacked.value_counts(), columns=['Count']).reset_index()
    cat_count1 = []

    for i in range (len(cat_count)):
        cat_count1.append(cat_count['index'][i].lstrip())
    
    cat_name = list(dict.fromkeys(cat_count1))
    return cat_name

In [248]:
print(cat_name(df1))

['R&D', 'Laboratory ', 'Information Technology', 'Sciences ', 'Engineering', 'Finance', 'Banking ', 'Civil Service', 'Public ', 'Consulting', 'Telecommunications', 'Others', 'Manufacturing', 'Education ', 'Pharmaceutical', 'Healthcare ', 'Training', 'Insurance', 'General Management', 'Risk Management', 'Logistics ', 'Supply Chain']


In [249]:
def clean_kie(df):

    #extract only number from string
    df['Year_Experience'] = df['Year_Experience'].str.extract('(\d+)')
    
    #remove comma from cell with string
    clean_list = "(,|;|â||¦|®|)"
    for col in df.columns.difference(['Year_Experience','Min_Salary','Max_Salary','Avg_Salary']):
        df[col]=df[col].str.replace(clean_list, "")
        
    #remove extra whitespace between string
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    #fill NaN in year of experience with 0
    df['Year_Experience'] = df['Year_Experience'].fillna(0)
    return df

In [298]:
df_clean = clean_kie(df1)
print(df_clean.shape)
df_clean.head(2)

(453, 13)


,Job_Id,Emp_Type,Job_Title,Company,Date_Posted,Year_Experience,Seniority,Category,Requirements,Min_Salary,Max_Salary,Avg_Salary,Salary_range
0,MCF-2020-0045240,Contract Permanent,Scientist (Machine Intellection) I2R,A*STAR RESEARCH ENTITIES,Posted 27 Mar 2020,0,Professional,Sciences Laboratory R&D,Developing enhancing automating and managing a...,4500.0,9000.0,6750.0,Med
1,MCF-2020-0045289,Contract Permanent,Scientist (Machine Intellection) I2R,A*STAR RESEARCH ENTITIES,Posted 27 Mar 2020,0,Professional,Sciences Laboratory R&D,Protein nodes discovering in cancer Biomarker ...,4500.0,9000.0,6750.0,Med


In [299]:
#word count function
def word_count(df_col):

    str_counts = 0
    sum_str = 0

    for i in range (len(df_col)):    
        str_counts = len(df_col[i].split())
        sum_str = sum_str + str_counts

    print(sum_str)

In [300]:
#number of word found in Requirements column before clean
word_count(df_clean['Requirements'])

78351


In [301]:
def freq_words(word_count, features):

    num_word = np.asarray(word_count.sum(axis=0)).reshape(-1)
    most_count = num_word.argsort()[::-1]
    key_word = pd.Series(num_word[most_count], 
                           index=features[most_count])

    return key_word

In [302]:
def stop_word_fil(df):
    
    #stop words were added to filter some generic recurring business terms.
    stop = stopwords.words('english')
    stop += ['regret','shortlisted', 'candidates','notified','etc', 'take', 'hands','added','able','writting',
             'year','years','least', 'related','using', 'and', 'ability','work','skills','advantage','written'
            'develop','good','team','design','knowledge','experience','following','areas', 'ability','and','in','to']
    
    #most common words for requirements
    cvt = CountVectorizer(lowercase=True, strip_accents='unicode',max_features=80000, min_df=1, max_df=0.9,
                          stop_words=stop, ngram_range=(1,2))
    vect_word = cvt.fit_transform(df['Requirements'])
    features = np.array(cvt.get_feature_names()) 

    key_word = freq_words(vect_word, features)
    
    #update stop_word with common words
    new_stop = key_word[key_word<5].index
    stop.extend(new_stop)
    
    pat = r'\b(?:{})\b'.format('|'.join(stop))
    df['Requirements'] = df['Requirements'].str.replace(pat, " ")
    df['Requirements'] = df['Requirements'].map(lambda x: x.strip())
    df['Requirements'] = df['Requirements'].replace({' +':" "},regex=True)
    
    return df

In [303]:
data_df = stop_word_fil(df_clean)

#number of word found in Requirements column after clean
word_count(data_df['Requirements'])

39378


In [307]:
data_df.to_csv('JOB_DATA_v10.csv', index=False, encoding='utf-8')

In [304]:
data_df2 = data_df.copy()

In [305]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [306]:
for i in range (len(data_df2['Requirements'])):
    if (isEnglish(data_df2['Requirements'][i]) == False):
        print(data_df2['Requirements'][i], i)

You   lead designing processes   analyse structured unstructured data   sources insights Collaborate   services subject matter experts (SMEs) drive   analytical You   enable ensuring   applying data science  artificial intelligence Identify  new data sources multiple   findings analysis   experiments IBM teams stakeholders Develop models analysis across range Quote Cash functions Design   tracking actions   analyse impact business outcomes implementation   deployment analytics Design efficient Spark Monitor Spark cluster performance A  Degree quantitative field (Statistics Mathematics Operations  Research Economics Bioinformatics Computer Science ). A   research quantitative field  plus. 3-5 working data sets models Strong statistical Excellent  understanding machine learning techniques algorithms  Logistic Regression Decision Trees Random Forrest Support Vector  Machines Clustering Expertise   various data ( .e. CRISP-DM  & engineering well  evaluation validation) Experience   librari

## Machine Learning

In [ ]:
#Dummified Seniority columns to use as predictor features
seniority_cat=data_df['Seniority'].str.get_dummies()
salary_cat=data_df['Salary_range'].str.get_dummies()
df = pd.concat([data_df, seniority_cat, salary_cat], axis=1)

In [ ]:
#CountVectorizer job requirements columns
stop = stopwords.words('english')

cvec = CountVectorizer(lowercase=True, strip_accents='unicode',
                       max_features=1500, min_df=5, max_df=0.7, 
                       stop_words=stop,ngram_range=(2,3))
cvec.fit(df['Requirements'])

In [ ]:
df.head(1)

In [ ]:
#creating predictor and target dataset
model_data = df.drop(columns=['Job_Title','Company','Seniority','Category','Min_Salary',
                               'Max_Salary','Emp_Type','Avg_Salary', 'Job_Id', 'Date_Posted'])

nlp = pd.DataFrame(cvec.transform(model_data['Requirements']).todense(),columns=cvec.get_feature_names())

senior_nlp = pd.concat([model_data, nlp], axis=1)

In [ ]:
X = senior_nlp.drop(columns=['Salary_range','Requirements'])
X_nlp = nlp
y = senior_nlp['Salary_range'].values
X.shape

In [ ]:
#Data with dummified 'seniority' and countvectorized 'requirements'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#Data with countvectorized 'requirements' only
X_train_nlp, X_test_nlp, y_train_nlp, y_test_nlp = train_test_split(X_nlp, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier(max_depth=4, random_state=42)
dtc = dtc.fit(X_train , y_train)

dtc1 = DecisionTreeClassifier(max_depth=4, random_state=42)
dtc_nlp = dtc1.fit(X_train_nlp , y_train_nlp)

In [ ]:
print(classification_report(y_test,dtc.predict(X_test),target_names=["Low", "Med", "High"]))

In [ ]:
pd.DataFrame(confusion_matrix(y_test,dtc.predict(X_test)),
             index=['Actual Low','Actual Med', 'Actual High'],
             columns=['Pred Low','Pred Med','Pred High'])

In [ ]:
print(classification_report(y_test_nlp,dtc_nlp.predict(X_test_nlp),target_names=["Low", "Med", "High"]))

In [ ]:
pd.DataFrame(confusion_matrix(y_test_nlp,dtc_nlp.predict(X_test_nlp)),
             index=['Actual Low','Actual Med', 'Actual High'],
             columns=['Pred Low','Pred Med','Pred High'])